# Analytical Regression
Computing a regression using preudo-inverse matrices

Author: Pierre Nugues

## The modules

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## The Functions

### Pseudo-inverse with a numpy `matrix`

In [ ]:
def regression_matrix(X, y, reg=0.0):
    """
    Computes the regression using numpy matrices
    :param observations, regularization
    :return: weights, ŷ, se, sse
    """
    if reg != 0.0:
        print('Regularized')
    I = np.identity(X.shape[1])
    w = (X.T * X + reg * I).I * X.T * y
    y_hat = X * w
    se = np.square(y_hat - y)
    sse = (y_hat - y).T * (y_hat - y)
    return w, y_hat, se, sse

### Pseudo-inverse with a numpy `array`

In [ ]:
def regression_array(X, y, reg=0.0):
    """
    Computes the regression using numpy arrays
    :param observations:
    :return: weights, ŷ, sse
    """
    if reg != 0.0:
        print('Regularized')
    I = np.identity(X.shape[1])
    w = (np.linalg.inv(X.T @ X + reg * I) @ X.T) @ y
    # Or directly with pinv()
    # w = np.linalg.pinv(X) @ y
    y_hat = X @ w
    se = (y_hat - y) * (y_hat - y)
    sse = (y_hat - y).T @ (y_hat - y)
    return w, y_hat, se, sse

## The Dataset

The number of characters and number of _a_ in the French and English chapters

In [ ]:
stat_fr = np.array([[36961, 2503],
                      [43621, 2992],
                      [15694, 1042],
                      [36231, 2487],
                      [29945, 2014],
                      [40588, 2805],
                      [75255, 5062],
                      [37709, 2643],
                      [30899, 2126],
                      [25486, 1784],
                      [37497, 2641],
                      [40398, 2766],
                      [74105, 5047],
                      [76725, 5312],
                      [18317, 1215]])

stat_en = np.array([[35680, 2217],
                      [42514, 2761],
                      [15162, 990],
                      [35298, 2274],
                      [29800, 1865],
                      [40255, 2606],
                      [74532, 4805],
                      [37464, 2396],
                      [31030, 1993],
                      [24843, 1627],
                      [36172, 2375],
                      [39552, 2560],
                      [72545, 4597],
                      [75352, 4871],
                      [18031, 1119]])

In [ ]:
stat_fr.shape

In [ ]:
np.ones((stat_fr.shape[0],1))

In [ ]:
stat_fr = np.hstack((np.ones((stat_fr.shape[0],1)), stat_fr))
stat_fr

In [ ]:
stat_en = np.hstack((np.ones((stat_en.shape[0],1)), stat_en))
stat_en

## Computing the Regression Using `array`

In [ ]:
pattern = [('red', 's'), ('green', '^')]
lang = [None] * 2

In [ ]:
for i, stats in enumerate([stat_en, stat_fr]):
    x_l = stats[:, 1]
    y_l = stats[:, -1]
    lang[i] = plt.scatter(x_l, y_l, color=pattern[i][0], marker=pattern[i][1])
    X = stats[:, :-1]
    y =stats[:, -1:]
    w, y_hat, se, sse = regression_array(X, y)
    print('Language:', i)
    print('X:', X)
    print('y:', y)
    print('ŷ:', y_hat)
    print('Squared errors:', se)
    print("Weights", w.T)
    print("SSE", sse)
    plt.plot([min(x_l), max(x_l)],
             [([1, min(x_l)] @ w), ([1, max(x_l)] @ w)],
             color=pattern[i][0])
plt.title("Salammbô")
plt.xlabel("Letter count")
plt.ylabel("A count")
plt.legend((lang[0], lang[1]), ('English', 'French'), loc='lower right', scatterpoints=1)
plt.show()

## Computing the Regression Using `matrix`

In [ ]:
for i, stats in enumerate([stat_en, stat_fr]):
    x_l = stats[:, 1]
    y_l = stats[:, -1]
    lang[i] = plt.scatter(x_l, y_l, color=pattern[i][0], marker=pattern[i][1])
    X = np.matrix(stats[:, :-1])
    y = np.matrix(stats[:, -1:])
    w, y_hat, se, sse = regression_matrix(X, y)
    print('Language:', i)
    print('X:', X)
    print('y:', y)
    print('ŷ:', y_hat)
    print('Squared errors:', se)
    print("Weights", w.T)
    print("SSE", sse)

    w = np.array(w)
    plt.plot([min(x_l), max(x_l)],
             [([1, min(x_l)] @ w), ([1, max(x_l)] @ w)],
             color=pattern[i][0])
plt.title("Salammbô")
plt.xlabel("Letter count")
plt.ylabel("A count")
plt.legend((lang[0], lang[1]), ('English', 'French'), loc='lower right', scatterpoints=1)
plt.show()

## Singular Matrix

We now introduce a singular matrix, something all too frequent in regression experiments. We just duplicate one column.

In [ ]:
stat_fr

In [ ]:
stat_fr_sing = np.hstack((stat_fr[:, :-1], stat_fr[:, -2:-1], stat_fr[:, -1:]))
stat_fr_sing

In [ ]:
print('Trying regularization with a singular matrix')
# Creation of a singular matrix by duplicating a column
X = np.array(stat_fr_sing)[:, :-1]
y = np.array(stat_fr_sing)[:, -1]
print('X:', X)
print('y:', y)
try:
    regression_array(X, y)
except:
    print(np.linalg.linalg.LinAlgError)
    print("Singular matrix: Could not be inverted.")

### Singular Matrix with Regularization

Returns $\mathbf{w}$, $\mathbf{\hat{y}}$, squared errors, and the sum of squared errors

In [ ]:
w, y_hat, se, sse = regression_array(X, y, reg=0.01)
w, y_hat, se, sse

### Pseudo-inverse with a Quasisingular Matrix

We now try regularization with a quasi singular matrix

In [ ]:
print('Trying regularization with a quasi singular matrix')
np.set_printoptions(precision=10)
X = np.array(stat_fr_sing)[:, :-1]
y = np.array(stat_fr_sing)[:, -1]
X, y

In [ ]:
X[0][2] -= 0.000001
X, y

Even if the matrix is not mathematically singular, we have an unstable result with very high weights and an astronomic loss.

In [ ]:
# No regularization
w, y_hat, se, sse = regression_array(X, y)
w, y_hat, se, sse

### With Regularization

A small regularization adds stability and results in a loss that is the same as with a nonsingular matrix

In [ ]:
# With regularization
w, y_hat, se, sse = regression_array(X, y, reg=0.01)
w, y_hat, se, sse